In [24]:
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from torchvision.transforms import functional as F
import os
import glob

In [39]:
input_base_folder = '../../data/classification/raw'
output_base_folder = '../../data/classification/preprocessed'

In [9]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, num_classes=91)

device = torch.device("cuda") if False else torch.device("cpu")
checkpoint = torch.load('../../model/detection.pth', map_location=device)
model.load_state_dict(checkpoint)
model.eval()

c:\Users\Administrator\Documents\spark-plug-classification\env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Administrator\Documents\spark-plug-classification\env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_7960\200923494.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more 

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu

In [46]:
def process_folder(input_folder, output_folder):
    image_extensions = ['*.jpg', '*.JPG', '*.png', '*.PNG']

    image_paths = []
    for ext in image_extensions:
        image_paths.extend(glob.glob(os.path.join(input_folder, ext)))

    for image_path in image_paths:
        image = Image.open(image_path)

        if image.mode != 'L':
            image = image.convert('L')

        image_np = np.array(image)
        image_tensor = F.to_tensor(image_np)
        image_tensor = image_tensor.unsqueeze(0)

        with torch.no_grad():
            predictions = model(image_tensor)

        boxes = predictions[0]['boxes'].cpu().numpy() 
        scores = predictions[0]['scores'].cpu().numpy()
        labels = predictions[0]['labels'].cpu().numpy() 

        for i, (box, score, label) in enumerate(zip(boxes, scores, labels)):
            if score > 0.5: 
                x1, y1, x2, y2 = map(int, box)

                cropped_image = image.crop((x1, y1, x2, y2))

                relative_path = os.path.relpath(image_path, input_folder) 
                folder_name = os.path.dirname(relative_path)

                output_subfolder = os.path.join(output_folder, folder_name)
                os.makedirs(output_subfolder, exist_ok=True)

                image_name = os.path.basename(image_path)
                output_name = f'{os.path.splitext(image_name)[0]}.jpg'
                output_path = os.path.join(output_subfolder, output_name)

                cropped_image.save(output_path)
                print(f"Gambar disimpan di: {output_path}")
                # break (Jika tidak mau mendapat hasil deteksi terakhir bisa di uncomment)

In [47]:
for root, dirs, files in os.walk(input_base_folder):
    for dir_name in dirs:
        input_folder = os.path.join(root, dir_name) 
        output_folder = os.path.join(output_base_folder, os.path.relpath(input_folder, input_base_folder))

        os.makedirs(output_folder, exist_ok=True)

        process_folder(input_folder, output_folder)

Gambar disimpan di: ../../data/classification/preprocessed\boros(kaya)\IMG_5373.jpg
Gambar disimpan di: ../../data/classification/preprocessed\boros(kaya)\IMG_5378.jpg
Gambar disimpan di: ../../data/classification/preprocessed\boros(kaya)\IMG_5379.jpg
Gambar disimpan di: ../../data/classification/preprocessed\boros(kaya)\IMG_5373.jpg
Gambar disimpan di: ../../data/classification/preprocessed\boros(kaya)\IMG_5378.jpg
Gambar disimpan di: ../../data/classification/preprocessed\boros(kaya)\IMG_5379.jpg
Gambar disimpan di: ../../data/classification/preprocessed\boros(kaya)\IMG_5346.jpg
Gambar disimpan di: ../../data/classification/preprocessed\boros(kaya)\IMG_5347.jpg
Gambar disimpan di: ../../data/classification/preprocessed\boros(kaya)\IMG_5348.jpg
Gambar disimpan di: ../../data/classification/preprocessed\boros(kaya)\IMG_5349.jpg
Gambar disimpan di: ../../data/classification/preprocessed\boros(kaya)\IMG_5349.jpg
Gambar disimpan di: ../../data/classification/preprocessed\boros(kaya)\IMG_5